In [2]:
import multiprocessing
import inspect
import tempfile
import sys
import pickle

import matplotlib
import matplotlib.pyplot as plt

from importlib import import_module, invalidate_caches
from functools import partial
from random import randint
from collections import Counter

from bs4 import BeautifulSoup
from tqdm.auto import tqdm, trange

from pyecsca.ec.params import DomainParameters, get_params
from pyecsca.ec.mult import *
from pyecsca.sca.re.rpa import MultipleContext, rpa_distinguish, RPA, multiples_computed
from pyecsca.ec.context import DefaultContext, local
from pyecsca.ec.model import ShortWeierstrassModel
from pyecsca.ec.coordinates import AffineCoordinateModel
from pyecsca.misc.utils import TaskExecutor

In [ ]:
model = ShortWeierstrassModel()

# All dbl-and-add multipliers from https://github.com/J08nY/pyecsca/blob/master/pyecsca/ec/mult

# Use partial funcs such that the "multiples_computed" method from RPA module can use fake formulas.
window_mults = [
    partial(SlidingWindowMultiplier, width=4),
    partial(SlidingWindowMultiplier, width=5),
    partial(SlidingWindowMultiplier, width=6),
    partial(FixedWindowLTRMultiplier, m=2**4),
    partial(FixedWindowLTRMultiplier, m=2**5),
    partial(FixedWindowLTRMultiplier, m=2**6),
    partial(WindowBoothMultiplier, width=4),
    partial(WindowBoothMultiplier, width=5),
    partial(WindowBoothMultiplier, width=6)
]
naf_mults = [
    partial(WindowNAFMultiplier, width=4),
    partial(WindowNAFMultiplier, width=5),
    partial(WindowNAFMultiplier, width=6),
    partial(BinaryNAFMultiplier)
]
comb_mults = [
    partial(CombMultiplier, width=4),
    partial(CombMultiplier, width=5),
    partial(CombMultiplier, width=6),
    partial(BGMWMultiplier, width=4),
    partial(BGMWMultiplier, width=5),
    partial(BGMWMultiplier, width=6)
]
binary_mults = [
    partial(LTRMultiplier),
    partial(RTLMultiplier),
    partial(CoronMultiplier)
]
other_mults = [
    partial(SimpleLadderMultiplier),
    partial(FullPrecompMultiplier)
]

with_precomputation = window_mults + naf_mults[:-1] + other_mults[:-1] + comb_mults

all_mults = window_mults + naf_mults + binary_mults + other_mults + comb_mults

In [3]:
def get_small_scalars(params, mult, scalar, precomp_only = False):
    mult_class = mult.func
    if precomp_only:
        use_init = True
        use_multiply = False
    else:
        use_init = True
        use_multiply = True
    return multiples_computed(scalar, params, mult_class, mult, use_init, use_multiply)

def divides_any(l,small_scalars):
    for s in small_scalars:
        if s%l==0:
            return True
    return False

def mult_label(mult):
    if isinstance(mult, ScalarMultiplier):
        for attr in ("width", "m"):
            if not hasattr(mult, attr):
                continue
            return f"{mult.__class__.__name__}_{getattr(mult, attr)}"
        return mult.__class__.__name__
    else:
        # mult is a callable created from partial()
        return f"{mult.func.__name__}_{mult.args}_{mult.keywords}"

def get_general_distributions(divisors, bits, samples = 1000):
    distributions = {l:0 for l in divisors}
    for _ in range(samples):
        big_scalar = randint(1,2**bits)
        for l in divisors:
            if big_scalar%l==0:
                distributions[l]+=1
    for l,v in distributions.items():
        distributions[l] = v/samples
    return distributions

def get_general_n_distributions(divisors, bits, n, samples = 1000):
    distributions = {l:0 for l in divisors}
    for _ in range(samples):
        big_scalars = []
        for i in range(n):
            b = randint(1,256)
            
            big_scalars.append(randint(2**b,2**(b+1)))
        for l in divisors:
            if divides_any(l, big_scalars):
                distributions[l]+=1
    for l,v in distributions.items():
        distributions[l] = v/samples
    return distributions

def get_small_scalar_distributions(mult, category, curve, divisors, bits, samples = 1000, precomp_only = False):
    small_scalars_distributions = {l:0 for l in divisors}
    params = get_params(category, curve, "projective")
    for _ in range(samples):
        big_scalar = randint(1,2**bits)
        small_scalars = get_small_scalars(params, mult, big_scalar, precomp_only)
        for l in divisors:
            if divides_any(l, small_scalars):
                small_scalars_distributions[l]+=1
    for l,v in small_scalars_distributions.items():
        small_scalars_distributions[l] = v/samples
    return small_scalars_distributions

def merge_probs(*prob_maps):
    # Merge two or more maps of "small-scalar" -> "probability" together by averaging them.
    # This is correct if they were collected with the same amount of samples. If the
    # amount of samples differs a lot this will not update as much as it should, but will
    # update in the correct direction nonetheless.
    counter = Counter()
    nprobs = len(prob_maps)
    for prob_map in prob_maps:
        for k, v in prob_map.items():
            counter[k] += v
    return {k: v / nprobs for k, v in counter.items()}

In [ ]:
#2<p<200
small_primes = [3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199]
#200<p<400

medium_primes = [211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397]

powers_of_two = [2, 4, 8, 16, 32, 64, 128, 256, 512]

all_divisors = small_primes+medium_primes+powers_of_two


In [ ]:
category = "secg"
curve = "secp256r1"
num_workers = 16
bits = 256
samples = 10000
selected_mults = all_mults
selected_divisors = all_divisors

distributions_mults = {}
distributions_mults_precomp = {}

In [ ]:
with TaskExecutor(max_workers=num_workers) as pool:
    for mult in selected_mults:
        pool.submit_task(mult,
                         get_small_scalar_distributions,
                         mult, category, curve, selected_divisors, bits, samples)
    for mult, future in tqdm(pool.as_completed(), desc="Computing small scalar distributions.", total=len(pool.tasks)):
        print(f"Got {mult_label(mult)}.")
        if mult not in distributions_mults:
            distributions_mults[mult] = future.result()
        else:
            # Accumulate
            distributions_mults[mult] = merge_probs(distributions_mults[mult], future.result())

with TaskExecutor(max_workers=num_workers) as pool:
    for mult in with_precomputation:
        pool.submit_task(mult,
                         get_small_scalar_distributions,
                         mult, category, curve, selected_divisors, bits, samples, precomp_only=True)
    for mult, future in tqdm(pool.as_completed(), desc="Computing small scalar distributions (precomp_only).", total=len(pool.tasks)):
        print(f"Got {mult_label(mult)}.")
        if mult not in distributions_mults_precomp:
            distributions_mults_precomp[mult] = future.result()
        else:
            # Accumulate
            distributions_mults_precomp[mult] = merge_probs(distributions_mults_precomp[mult], future.result())

# Single-core variant:
# distributions_mults = {mult:get_small_scalar_distributions(mult,category,curve,selected_divisors,bits, samples) for mult in tqdm(selected_mults)}
# distributions_mults_precomp = {mult:get_small_scalar_distributions(mult,category,curve,selected_divisors,bits,samples,precomp_only=True) for mult in with_precomputation}

# Dump
with open(f"distributions_{category}_{curve}_{bits}","wb") as h:
    pickle.dump(distributions_mults, h)
with open(f"distributions_{category}_{curve}_{bits}_precomp","wb") as h:
    pickle.dump(distributions_mults_precomp, h)

# Load
#with open(f"distributions_{category}_{curve}_{bits}","rb") as h:
#    distributions_mults = pickle.load(h)
#with open(f"distributions_{category}_{curve}_{bits}_precomp","rb") as h:
#    distributions_mults_precomp = pickle.load(h)

In [ ]:
# general_distributions = get_general_distributions(selected_divisors, bits, samples)
# general_n_distributions = get_general_n_distributions(selected_divisors, bits, 256, samples)

In [ ]:

selected_mults = all_mults#window_mults[0:1]+window_mults[5:6]+naf_mults[1:2]#[mult for mult in all_mults if not mult in comb_mults]
selected_divisors = all_divisors
colors = {mult:matplotlib.cm.tab20(range(len(selected_mults)))[i] for i,mult in enumerate(selected_mults)}


fig = plt.subplots(figsize =(36, 12)) 

L = len(selected_divisors)
selected_divisors = sorted(selected_divisors)
for mult in selected_mults:
    y_values = [distributions_mults[mult][l] for l in selected_divisors]
    plt.plot([l for l in range(L)],y_values,color = colors[mult], label = mult_label(mult))

# mult = list(fixedwindow_dist.keys())[0]
# plt.plot([l for l in range(L)],[fixedwindow_dist[l] for l in selected_divisors],color = "pink", label = mult_label(mult))

# measured_dist = measured_distribution(library,selected_divisors)
# mes_x, mes_y = [],[]
# for i,l in enumerate(selected_divisors):
#     if l in measured_dist:
#         mes_y.append(measured_dist[l])
#         mes_x.append(i)
# plt.scatter(mes_x,mes_y,color = "black", label = library)

attempts = 0
fails =0
for i in range(51):
    with open(f"cards/jcop/199_{i}.txt") as f:
        attempts += f.read().count("ALG_EC_SVDP_DH of remote pubkey and local privkey")+1
        fails += 1
plt.scatter([selected_divisors.index(199)],[fails/attempts],s=[40],color = "black", label = "jcop")

plt.plot([l for l in range(L)],[general_distributions[l] for l in selected_divisors],color = "black", label = "prime-distribution")


plt.xlabel('divisors') 
plt.ylabel("prob") 
plt.xticks([r for r in range(L)], selected_divisors)

plt.legend()
plt.show() 
fig[0].savefig(f"graphs/re.png",dpi=300)


In [ ]:
selected_mults = with_precomputation
selected_divisors = small_primes#all_divisors
colors = {mult:matplotlib.cm.tab20(range(len(selected_mults)))[i] for i,mult in enumerate(selected_mults)}


fig = plt.subplots(figsize =(24, 12)) 

L = len(selected_divisors)
selected_divisors = sorted(selected_divisors)
for mult in selected_mults:
    plt.plot([l for l in range(L)],[distributions_mults_precomp[mult][l] for l in selected_divisors],color = colors[mult], label = mult_label(mult))


measured_dist = measured_distribution(library,selected_divisors)
mes_x, mes_y = [],[]
for i,l in enumerate(selected_divisors):
    if l in measured_dist:
        mes_y.append(measured_dist[l])
        mes_x.append(i)
plt.scatter(mes_x,mes_y,color = "black", label = library)


plt.xlabel('divisors') 
plt.ylabel("prob") 
plt.xticks([r for r in range(L)], selected_divisors)

plt.legend()
plt.show() 

In [ ]:
def nok_ecdh(line):
    return int(line.split(";")[-1].strip(),16)==0

def measured_distribution(library, selected_divisors):
    measured_distribution = {}
    counts = {order:0 for order in selected_divisors}
    for div in selected_divisors:
        errors = 0
        with open(f"./ecdh/{library}/ecdh_{div}.txt") as f:
            for line in f.readlines()[1:]:
                if nok_ecdh(line):
                    errors+=1
                counts[div]+=1
        measured_distribution[div] = errors
    
    for o,v in measured_distribution.items():
        if counts[o]!=0:
            measured_distribution[o] = v/counts[o]
    return measured_distribution

In [ ]:
selected_mults = other_mults[:1]+binary_mults[:1]+comb_mults[:1]+window_mults[:1]
selected_divisors = small_primes#all_divisors
library = "tomcrypt"
colors = {mult:matplotlib.cm.tab20(range(len(selected_mults)))[i] for i,mult in enumerate(selected_mults)}

fig = plt.subplots(figsize =(24, 12)) 

L = len(selected_divisors)
selected_divisors = sorted(selected_divisors)
for mult in selected_mults:
    plt.plot([l for l in range(L)],[distributions_mults[mult][l] for l in selected_divisors],color = colors[mult], label = mult_label(mult))


measured_dist = measured_distribution(library,selected_divisors)
mes_x, mes_y = [],[]
for i,l in enumerate(selected_divisors):
    if l in measured_dist:
        mes_y.append(measured_dist[l])
        mes_x.append(i)
plt.scatter(mes_x,mes_y,color = "black", label = library)


plt.xlabel('divisors') 
plt.ylabel("prob") 
plt.xticks([r for r in range(L)], selected_divisors)
plt.legend(loc="upper right")
plt.show() 
fig[0].savefig(f"graphs/{library}/re.png",dpi=300)

In [ ]:
selected_mults = [mult for mult in with_precomputation if mult in comb_mults]
selected_divisors = small_primes#all_divisors
library = "mbedtls"
colors = {mult:matplotlib.cm.tab20(range(len(selected_mults)))[i] for i,mult in enumerate(selected_mults)}

fig = plt.subplots(figsize =(24, 12)) 

L = len(selected_divisors)
selected_divisors = sorted(selected_divisors)
for mult in selected_mults:
    plt.plot([l for l in range(L)],[distributions_mults_precomp[mult][l] for l in selected_divisors],color = colors[mult], label = mult_label(mult))


measured_dist = measured_distribution(library,selected_divisors)
mes_x, mes_y = [],[]
for i,l in enumerate(selected_divisors):
    if l in measured_dist:
        mes_y.append(measured_dist[l])
        mes_x.append(i)
plt.scatter(mes_x,mes_y,color = "black", label = library)

plt.xlabel('divisors') 
plt.ylabel("prob") 
plt.xticks([r for r in range(L)], selected_divisors)
plt.legend()
plt.show() 
fig[0].savefig(f"graphs/{library}/re.png",dpi=300)

BouncyCastle
 - WindowBooth-5?

Mbedtls
 - CombMultiplier-4
 - confirmed in library
   
tomcrypt
 - ladder or coron
 - ladder confirmed in library

OpenSSL, LibreSSL, Botan, Crypto++ and IPPCP followed the general distribution of divisibility by the primes. So they have some countermeasure.

Note that some libraries for some orders output "Invalid algorithm parameter: Not supported.". For libcrypt, SunEC and Nettle it happened for all orders.
BoringSSL
 - "Invalid algorithm parameter: Error creating EC_GROUP, EC_GROUP_set_generator."


In [ ]:
def scatter(library, color):
    measured_dist = measured_distribution(library,selected_divisors)
    mes_x, mes_y = [],[]
    for i,l in enumerate(selected_divisors):
        if l in measured_dist:
            mes_y.append(measured_dist[l])
            mes_x.append(i)
    plt.scatter(mes_x,mes_y,color = color, label = library)

selected_divisors = small_primes#all_divisors

fig = plt.subplots(figsize =(24, 12)) 

L = len(selected_divisors)

colors = matplotlib.cm.tab20(range(6))
scatter("openssl",colors[0])
scatter("libressl",colors[1])
scatter("botan",colors[2])
scatter("Crypto++",colors[3])
scatter("ippcp",colors[4])

plt.plot([l for l in range(L)],[general_distributions[l] for l in selected_divisors],color = colors[5], label = "prime-distribution")

plt.xlabel('divisors') 
plt.ylabel("prob") 
plt.xticks([r for r in range(L)], selected_divisors)
plt.legend()
plt.show() 
fig[0].savefig(f"graphs/resistant.png",dpi=300)

In [ ]:
def scatter(library, color):
    measured_dist = measured_distribution(library,selected_divisors)
    mes_x, mes_y = [],[]
    for i,l in enumerate(selected_divisors):
        if l in measured_dist:
            mes_y.append(measured_dist[l])
            mes_x.append(i)
    # plt.scatter(mes_x,mes_y,color = color, label = library)
    plt.plot(mes_x,mes_y,color = color, linewidth = 1, label = library)

selected_divisors = small_primes[:12]#all_divisors

fig = plt.subplots(figsize =(10, 4)) 

L = len(selected_divisors)

colors = matplotlib.cm.tab20(range(9))
scatter("openssl",colors[0])
scatter("libressl",colors[1])
scatter("botan",colors[2])
scatter("Crypto++",colors[3])
scatter("mbedtls",colors[4])
scatter("libressl",colors[5])
scatter("BouncyCastle",colors[6])
scatter("tomcrypt",colors[7])
scatter("ippcp",colors[8])


# plt.plot([l for l in range(L)],[general_distributions[l] for l in selected_divisors],color = colors[9], label = "divison-distribution")

plt.xlabel('Input point order',fontsize=15) 
plt.ylabel("Error rate",fontsize=15) 
plt.xticks([r for r in range(L)], [v if i%1==0 else "" for i,v in enumerate(selected_divisors)])
plt.legend(loc="center right",prop={'size': 11})
plt.tight_layout()
plt.show() 
fig[0].savefig(f"graphs/lib_dists.png",dpi=300)

In [ ]:
def bars(library, color,shift,width):
    measured_dist = measured_distribution(library,selected_divisors)
    mes_x, mes_y = [],[]
    for i,l in enumerate(selected_divisors):
        offset = width*shift
        if l in measured_dist:
            mes_y.append(measured_dist[l])
            mes_x.append(2*i+offset)
    plt.bar(mes_x,mes_y,width=0.1,color = color,align ='center', label = labels.get(library,library))

selected_divisors = small_primes[:12]#all_divisors

fig = plt.subplots(figsize =(10, 4)) 

L = len(selected_divisors)

colors = matplotlib.cm.tab20(range(9))
width = 0.2
for i,lib in enumerate(("openssl","libressl","botan","Crypto++","mbedtls","BouncyCastle","tomcrypt","ippcp")):
    bars(lib,colors[i],i,width)


plt.plot([2*l+4*width for l in range(L)],[general_distributions[l] for l in selected_divisors],color = colors[8], label = "expected distribution")

plt.xlabel('Input point order',fontsize=15) 
plt.ylabel("Error rate",fontsize=15) 
plt.xticks([2*r+4*width for r in range(L)], [v if i%1==0 else "" for i,v in enumerate(selected_divisors)])
plt.legend(loc="upper right",prop={'size': 11})
plt.tight_layout()
plt.show() 
fig[0].savefig(f"graphs/lib_dists.png",dpi=300)

In [ ]:
from math import sqrt

selected_mults = all_mults#window_mults[0:1]+window_mults[5:6]+naf_mults[1:2]#[mult for mult in all_mults if not mult in comb_mults]
selected_divisors = small_primes#ll_divisors
colors = {mult:matplotlib.cm.tab20(range(len(selected_mults)))[i] for i,mult in enumerate(selected_mults)}


fig = plt.subplots(figsize =(30, 20)) 

L = len(selected_divisors)
selected_divisors = sorted(selected_divisors)
for mult in selected_mults:
    y_values,y_values_mstd, y_values_pstd = [],[],[]
    for l in selected_divisors:
        p = distributions_mults[mult][l]
        y_values.append(1/p)
        y_values_mstd.append(1/p-sqrt((1-p)/p**2))
        y_values_pstd.append(1/p+sqrt((1-p)/p**2))
    plt.plot([l for l in range(L)],y_values,color = colors[mult], label = mult_label(mult))
    plt.fill_between([l for l in range(L)], y_values_mstd , y_values_pstd, alpha = 0.1, color = colors[mult])
    
# mult = list(fixedwindow_dist.keys())[0]
# plt.plot([l for l in range(L)],[fixedwindow_dist[l] for l in selected_divisors],color = "pink", label = mult_label(mult))

# measured_dist = measured_distribution(library,selected_divisors)
# mes_x, mes_y = [],[]
# for i,l in enumerate(selected_divisors):
#     if l in measured_dist:
#         mes_y.append(measured_dist[l])
#         mes_x.append(i)
# plt.scatter(mes_x,mes_y,color = "black", label = library)

plt.plot([l for l in range(L)],[1/general_distributions[l] for l in selected_divisors],color = "black", label = "prime-distribution")

steps_dist = {}
for i in range(51):
    with open(f"cards/jcop/199_{i}.txt") as f:
        steps = f.read().count("ALG_EC_SVDP_DH of remote pubkey and local privkey")
    if not steps in steps_dist:
        steps_dist[steps] = 0
    steps_dist[steps]+=1
ys = sorted(list(steps_dist.keys()))
plt.scatter([selected_divisors.index(199)]*len(ys),ys, s=[steps_dist[y]*20 for y in ys],color="black")
for i, y in enumerate(ys):
    plt.annotate(str(steps_dist[y]), (selected_divisors.index(199)-1, y))
avg = 0
for s,c in steps_dist.items():
    avg+=s*c
avg = avg/sum(steps_dist.values())
plt.scatter([selected_divisors.index(199)],[avg], s=[30],color="yellow")
    
plt.xlabel('divisors') 
plt.ylabel("prob") 
plt.yticks(range(12))
plt.xticks([r for r in range(L)], selected_divisors)

plt.legend()
plt.show() 
fig[0].savefig(f"graphs/re.png",dpi=300)